In [17]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [18]:
import pandas as pd
import numpy as np

1. 데이터 파일 불러오기

In [19]:
# 파일 불러오기
station = pd.read_csv('/content/gdrive/Shareddrives/서울시 빅데이터 캠퍼스 공모전/전처리 데이터/full_station.csv')
vacancy = pd.read_csv('/content/gdrive/Shareddrives/서울시 빅데이터 캠퍼스 공모전/전처리 데이터/공실.csv')
household = pd.read_csv('/content/gdrive/Shareddrives/서울시 빅데이터 캠퍼스 공모전/전처리 데이터/서울시_1인가구_4인가구(동별).csv')
business = pd.read_csv('/content/gdrive/Shareddrives/서울시 빅데이터 캠퍼스 공모전/전처리 데이터/서울시_산업체현황_종사자규모별_동별_처리.csv',encoding = 'euc-kr')
pop_dong = pd.read_csv('/content/gdrive/Shareddrives/서울시 빅데이터 캠퍼스 공모전/전처리 데이터/행정동_단위_거주인구.csv',encoding = 'euc-kr')

2. 지하철역 위치 전처리

In [20]:
# station 전처리 
# unnamed: 0, 도로명주소 열 삭제
station_info = station.drop(['Unnamed: 0','역사도로명주소'], axis = 1)
# 행정동 전처리
station_info['행정동'] = station_info['행정동'].str.split(' ').str[-1]
station_info.head()

,역사명,역위도,역경도,행정동
0,동대문역사문화공원역,37.565053,127.007243,광희동
1,신당역,37.565829,127.018102,황학동
2,상왕십리역,37.564171,127.029352,왕십리2동
3,왕십리역,37.561459,127.036640,행당1동
4,한양대역,37.555222,127.043492,사근동


3. 공실 데이터 전처리 및 지하철 역 위치 데이터와 통합

In [21]:
# 코드 다시 보니까 merge로 하면 간단할 것 같긴 한데 이미 한거니까 그냥 두겠삼 .. 

# 공실 데이터 추가
# 공실있는 지하철역
vc_station = vacancy['역명'].tolist()

for i in range(len(station_info)):
    h = station_info.loc[i,'역사명'] 
    # if vacancy 역명중에 있으면 공실여부 1, 공실_총_면적 추가
    if h in vc_station:
      station_info.loc[i,'공실여부'] = 1
      station_info.loc[i,'공실_총_면적'] = (vacancy[vacancy['역명'] == h].iloc[0])['공실 총 면적(m^2)']
    # 없으면, 공실여부 0, 공실_총_면적 0
    else:
      station_info.loc[i,'공실여부'] = 0
      station_info.loc[i,'공실_총_면적'] = 0

station_info.head()

,역사명,역위도,역경도,행정동,공실여부,공실_총_면적
0,동대문역사문화공원역,37.565053,127.007243,광희동,1.0,122.50
1,신당역,37.565829,127.018102,황학동,1.0,34.10
2,상왕십리역,37.564171,127.029352,왕십리2동,0.0,0.00
3,왕십리역,37.561459,127.036640,행당1동,1.0,19.00
4,한양대역,37.555222,127.043492,사근동,1.0,20.04


4. 동별 데이터 전처리
 
 4-1) 행정동별 가구수 (1인/4인가구)
 
 4-2) 행정동별 사업체수(1-4인/5-9인/9인이하)

 4-3) 행정동별 거주인구수
 
  - 반출데이터 33번 전처리 데이터 사용 
  - 전처리 코드는 공유드라이브/반출데이터/33번/파일데이터 에 있슴 !

 4-4) 행정동별 생활인구(유동인구) 
  
  - 전처리 되면 합치기


In [22]:
# 행정동 데이터끼리 묶기
# 1인/4인가구 + 9인이하 사업체
print('가구수 데이터 행정동 개수:', len(household['동']))
print('사업체수 데이터 행정동 개수:', len(business['행정동']))
print('거주인구 수 데이터 행정동 개수:', len(pop_dong['행정동명']))

가구수 데이터 행정동 개수: 425
사업체수 데이터 행정동 개수: 425
거주인구 수 데이터 행정동 개수: 422


In [23]:
# 행정동으로 열 이름 일치
household.columns = ['행정동', '1인가구', '4인가구']
household.head()

,행정동,1인가구,4인가구
0,가락1동,"1,681","2,481"
1,가락2동,"3,157","2,936"
2,가락본동,"4,770","1,762"
3,가리봉동,"4,139",250
4,가산동,"12,360",651


In [24]:
# 사업체수
business = business[['1-4 사업체수','5-9 사업체수','9인 이하\n사업체수']]
business.columns = ['1-4_사업체수','5-9_사업체수','9인이하_사업체수']
business.head()

,1-4_사업체수,5-9_사업체수,9인이하_사업체수
0,"2,451",448,"2,899"
1,"1,483",243,"1,726"
2,"2,613",496,"3,109"
3,917,44,961
4,"9,907","3,458","13,365"


In [25]:
# 거주인구수
pop_dong.columns = ['행정동','거주인구수']
pop_dong.head()

,행정동,거주인구수
0,가락1동,27473
1,가락2동,32575
2,가락본동,26564
3,가리봉동,8954
4,가산동,23377


In [26]:
# 행정동별 데이터 합치기
# 가구수 데이터랑 사업체수 데이터는 개수도 같고, 오름차순으로 정리되있어서 그냥 concat해도 됨
dong = pd.concat([household,business],axis=1)
dong

,행정동,1인가구,4인가구,1-4_사업체수,5-9_사업체수,9인이하_사업체수
0,가락1동,"1,681","2,481","2,451",448,"2,899"
1,가락2동,"3,157","2,936","1,483",243,"1,726"
2,가락본동,"4,770","1,762","2,613",496,"3,109"
3,가리봉동,"4,139",250,917,44,961
4,가산동,"12,360",651,"9,907","3,458","13,365"
...,...,...,...,...,...,...
420,효창동,"1,616",671,313,64,377
421,후암동,"4,101",899,773,107,880
422,휘경1동,"4,177",759,714,164,878
423,휘경2동,"5,923","1,525","1,087",82,"1,169"


In [33]:
# 거주인구수 데이터는 결측동이 있는 것 같으니 
# 행정동으로 일단 merge 한 다음에 결측치 노가다로 채워주자 .. ^^
dong_2 = pd.merge(dong, pop_dong, how = 'left', on = '행정동')
dong_2

,행정동,1인가구,4인가구,1-4_사업체수,5-9_사업체수,9인이하_사업체수,거주인구수
0,가락1동,"1,681","2,481","2,451",448,"2,899",27473.0
1,가락2동,"3,157","2,936","1,483",243,"1,726",32575.0
2,가락본동,"4,770","1,762","2,613",496,"3,109",26564.0
3,가리봉동,"4,139",250,917,44,961,8954.0
4,가산동,"12,360",651,"9,907","3,458","13,365",23377.0
...,...,...,...,...,...,...,...
420,효창동,"1,616",671,313,64,377,10157.0
421,후암동,"4,101",899,773,107,880,17054.0
422,휘경1동,"4,177",759,714,164,878,13658.0
423,휘경2동,"5,923","1,525","1,087",82,"1,169",24117.0


In [35]:
dong_2[dong_2['거주인구수'].isnull()]
# 상일동은 202106데이터 쓰기 > 38683
# 종로5.6가동은 중간에 점때문에 안나오는거 > 넣어주기 > 5121
# 항동은 데이터 아예 없음 > 어짜피 지하철역 없으니까 0 넣기

,행정동,1인가구,4인가구,1-4_사업체수,5-9_사업체수,9인이하_사업체수,거주인구수
185,상일동,"2,219","3,833",787,72,859,NaN
354,종로5·6가동,"2,402",172,"6,460",552,"7,012",NaN
399,항동,"1,506","1,384",0,0,0,NaN


In [37]:
dong_2.loc[185,'거주인구수'] = 38683
dong_2.loc[354,'거주인구수'] = 5121
dong_2.loc[399,'거주인구수'] = 0
dong_2.isnull().sum()

행정동          0
1인가구         0
4인가구         0
1-4_사업체수     0
5-9_사업체수     0
9인이하_사업체수    0
거주인구수        0
dtype: int64

5. 지하철역 데이터 + 동별 데이터 합쳐 !!!

In [49]:
integ = pd.merge(station_info, dong_2, on = '행정동', how = 'left')

In [50]:
print(len(integ))
print(len(integ['역사명'].unique()))
# 하나 추가됐자나 !!! 찾아서 없애버려 !!!

297
296


In [51]:
display(integ[integ.duplicated(['역사명'])])

,역사명,역위도,역경도,행정동,공실여부,공실_총_면적,1인가구,4인가구,1-4_사업체수,5-9_사업체수,9인이하_사업체수,거주인구수
125,압구정역,37.526686,127.02837,신사동,0.0,0.0,"2,799","1,126","2,671",918,"3,589",38727.0


In [52]:
display(integ[integ['행정동']=='신사동'])
# 124 신사동은 관악구 신사동이다 !!!
# 서울에 신사동이 3개나 있다 몰라따

,역사명,역위도,역경도,행정동,공실여부,공실_총_면적,1인가구,4인가구,1-4_사업체수,5-9_사업체수,9인이하_사업체수,거주인구수
124,압구정역,37.526686,127.02837,신사동,0.0,0.0,"9,025",939,"1,127",164,"1,291",38727.0
125,압구정역,37.526686,127.02837,신사동,0.0,0.0,"2,799","1,126","2,671",918,"3,589",38727.0


In [53]:
# 압구정역 중복 행 지우기
integ = integ.drop(integ.index[124])
display(integ[integ['행정동']=='신사동'])
len(integ)

,역사명,역위도,역경도,행정동,공실여부,공실_총_면적,1인가구,4인가구,1-4_사업체수,5-9_사업체수,9인이하_사업체수,거주인구수
125,압구정역,37.526686,127.02837,신사동,0.0,0.0,"2,799","1,126","2,671",918,"3,589",38727.0


296

In [54]:
# 숫자 사이 쉼표 삭제
# 1인/4인 가구, 사업체수 정수형으로 바꾸자
integ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 296
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   역사명        296 non-null    object 
 1   역위도        296 non-null    float64
 2   역경도        296 non-null    float64
 3   행정동        296 non-null    object 
 4   공실여부       296 non-null    float64
 5   공실_총_면적    296 non-null    float64
 6   1인가구       296 non-null    object 
 7   4인가구       296 non-null    object 
 8   1-4_사업체수   296 non-null    object 
 9   5-9_사업체수   296 non-null    object 
 10  9인이하_사업체수  296 non-null    object 
 11  거주인구수      296 non-null    float64
dtypes: float64(5), object(7)
memory usage: 30.1+ KB


In [63]:
integ = integ.replace(',','',regex = True)
integ[["1인가구","4인가구",'1-4_사업체수','5-9_사업체수','9인이하_사업체수']] \
  = integ[["1인가구","4인가구",'1-4_사업체수','5-9_사업체수','9인이하_사업체수']].apply(pd.to_numeric)
integ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296 entries, 0 to 296
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   역사명        296 non-null    object 
 1   역위도        296 non-null    float64
 2   역경도        296 non-null    float64
 3   행정동        296 non-null    object 
 4   공실여부       296 non-null    float64
 5   공실_총_면적    296 non-null    float64
 6   1인가구       296 non-null    int64  
 7   4인가구       296 non-null    int64  
 8   1-4_사업체수   296 non-null    int64  
 9   5-9_사업체수   296 non-null    int64  
 10  9인이하_사업체수  296 non-null    int64  
 11  거주인구수      296 non-null    float64
dtypes: float64(5), int64(5), object(2)
memory usage: 30.1+ KB


In [64]:
integ.head()

,역사명,역위도,역경도,행정동,공실여부,공실_총_면적,1인가구,4인가구,1-4_사업체수,5-9_사업체수,9인이하_사업체수,거주인구수
0,동대문역사문화공원역,37.565053,127.007243,광희동,1.0,122.50,2247,163,7422,716,8138,5210.0
1,신당역,37.565829,127.018102,황학동,1.0,34.10,3815,549,1770,115,1885,12523.0
2,상왕십리역,37.564171,127.029352,왕십리2동,0.0,0.00,2892,1093,670,113,783,16627.0
3,왕십리역,37.561459,127.036640,행당1동,1.0,19.00,2672,909,1067,187,1254,13935.0
4,한양대역,37.555222,127.043492,사근동,1.0,20.04,5785,449,814,186,1000,11950.0


In [65]:
# 엑셀 파일 / csv 파일 모두 '전처리 데이터' 폴더에 올려놨습미다
integ.to_excel('통합데이터.xlsx',index=False)